In [1]:
from jhora.utils import julian_day_number
from jhora.horoscope.chart.charts import rasi_chart, navamsa_chart
import jhora.const as const
import jhora.panchanga.drik as drik
from jhora.panchanga.drik import Place
from vedicastro.VedicAstro import VedicHoroscopeData
from timezonefinder import TimezoneFinder
from datetime import datetime
import pytz
import pandas as pd
import numpy as np

# === Основные объекты ===

planet_map = {
    "Sun": "Солнце", "Moon": "Луна", "Mars": "Марс", "Mercury": "Меркурий", "Jupiter": "Юпитер",
    "Venus": "Венера", "Saturn": "Сатурн", "Rahu": "Раху", "Ketu": "Кету",
    "Uranus": "Уран", "Neptune": "Нептун", "Pluto": "Плутон", "Chiron": "Хирон",
    "Asc": "Асцендент", "Syzygy": "Сизигия"
}

sign_map = {
    "Aries": "Овен", "Taurus": "Телец", "Gemini": "Близнецы", "Cancer": "Рак",
    "Leo": "Лев", "Virgo": "Дева", "Libra": "Весы", "Scorpio": "Скорпион",
    "Sagittarius": "Стрелец", "Capricorn": "Козерог", "Aquarius": "Водолей", "Pisces": "Рыбы"
}

nakshatra_map = {
"Ashwini": "Ашвини",
"Bharani": "Бхарани",
"Krittika": "Криттика",
"Rohini": "Рохини",
"Mrigashīrsha": "Мригашира",
"Ardra": "Ардра",
"Punarvasu": "Пунарвасу",
"Pushya": "Пушья",
"Āshleshā": "Ашлеша",
"Maghā": "Магха",
"PūrvaPhalgunī": "Пурва Пхалгуни",
"UttaraPhalgunī": "Уттара Пхалгуни",
"Hasta": "Хаста",
"Chitra": "Читра",
"Svati": "Свати",
"Vishakha": "Вишакха",
"Anuradha": "Анурадха",
"Jyeshtha": "Джьештха",
"Mula": "Мула",
"PurvaAshadha": "Пурва Ашадха",
"UttaraAshadha": "Уттара Ашадха",
"Shravana": "Шравана",
"Dhanishta": "Дхаништха",
"Shatabhisha": "Шатабхиша",
"PurvaBhādrapadā": "Пурва Бхадрапада",
"UttaraBhādrapadā": "Уттара Бхадрапада",
"Revati": "Ревати"
}

main_grahas = [
    "Асцендент", "Солнце", "Луна", "Марс", "Меркурий",
    "Юпитер", "Венера", "Сатурн", "Раху", "Кету"
]

rasi_order = ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион",
              "Стрелец", "Козерог", "Водолей", "Рыбы"]

sign_lords = {
    "Овен": "Марс", "Телец": "Венера", "Близнецы": "Меркурий", "Рак": "Луна",
    "Лев": "Солнце", "Дева": "Меркурий", "Весы": "Венера", "Скорпион": "Марс",
    "Стрелец": "Юпитер", "Козерог": "Сатурн", "Водолей": "Сатурн", "Рыбы": "Юпитер"
}

sign_order = {
    "Овен": 0, "Телец": 1, "Близнецы": 2, "Рак": 3,
    "Лев": 4, "Дева": 5, "Весы": 6, "Скорпион": 7,
    "Стрелец": 8, "Козерог": 9, "Водолей": 10, "Рыбы": 11
}

D1_TO_D9_MAP = {
    "Овен":      ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Телец":     ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Близнецы":  ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Рак":       ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
    "Лев":       ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Дева":      ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Весы":      ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Скорпион":  ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
    "Стрелец":   ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Козерог":   ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Водолей":   ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Рыбы":      ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
}

# === Вспомогательные функции ===

def get_birth_utc_offset_str(dt_obj, latitude, longitude):
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lat=latitude, lng=longitude)
    if not tz_name:
        raise ValueError("Не удалось определить часовой пояс для координат")
    tz = pytz.timezone(tz_name)
    localized_dt = tz.localize(dt_obj, is_dst=None)
    offset = localized_dt.utcoffset()
    hours = int(offset.total_seconds() // 3600)
    minutes = int((offset.total_seconds() % 3600) // 60)
    return f"{hours:+03d}:{abs(minutes):02d}"

def translate_chart_to_russian(df):
    df_translated = df.copy()
    for col, mapping in {
        "Планета": planet_map,
        "Знак": sign_map,
        "Накшатра": nakshatra_map,
        "Управитель": planet_map,
        "Управитель накшатры": planet_map
    }.items():
        if col in df_translated.columns:
            df_translated[col] = df_translated[col].map(mapping).fillna(df_translated[col])
    return df_translated

def filter_main_grahas(df):
    return df[df["Планета"].isin(main_grahas)].reset_index(drop=True)

def add_house_info(df):
    asc_row = df[df["Планета"] == "Асцендент"]
    if asc_row.empty:
        raise ValueError("В DataFrame нет строки с Асцендентом")
    asc_sign = asc_row.iloc[0]["Знак"]
    rasi_from_asc = rasi_order[rasi_order.index(asc_sign):] + rasi_order[:rasi_order.index(asc_sign)]
    sign_to_house = {sign: i + 1 for i, sign in enumerate(rasi_from_asc)}

    def planet_house(zodiac_sign):
        return sign_to_house.get(zodiac_sign, np.nan)

    def planet_rules(planet):
        ruled_signs = [sign for sign, lord in sign_lords.items() if lord == planet]
        ruled_houses = [sign_to_house[sign] for sign in ruled_signs]
        return ", ".join(str(h) for h in sorted(ruled_houses))

    df = df.copy()
    df["Дом"] = df["Знак"].apply(planet_house)
    df["Управляет домами"] = df["Планета"].apply(planet_rules)
    return df

# === Главная функция ===

def build_chart(birth_date: str, birth_time: str, latitude: str, longitude: str):
    dt = datetime.strptime(f"{birth_date} {birth_time}", "%d.%m.%Y %H:%M:%S")
    lat = float(latitude)
    lon = float(longitude)
    utc_offset = get_birth_utc_offset_str(dt, lat, lon)

    vhd = VedicHoroscopeData(
        year=dt.year, month=dt.month, day=dt.day,
        hour=dt.hour, minute=dt.minute, second=dt.second,
        utc=utc_offset, latitude=lat, longitude=lon,
        ayanamsa="Lahiri", house_system="Placidus"
    )

    vhd.get_ayanamsa()
    chart = vhd.generate_chart()
    planets_data = vhd.get_planets_data_from_chart(chart)
    df = pd.DataFrame(planets_data)

    # Определим колонку знака
    possible_sign_cols = ["Sign", "Rasi", "SignName"]
    sign_col = next((col for col in possible_sign_cols if col in df.columns), None)
    if sign_col is None:
        raise ValueError("Не найдена колонка с названием знака зодиака (Sign/Rasi/SignName)")

    columns_mapping = {
        "Object": "Планета",
        sign_col: "Знак",
        "Retrograde": "Ретроградность",
        "DMS": "Градусы",
        "Nakshatra": "Накшатра",
        "RasiLord": "Управитель",
        "NakshatraLord": "Управитель накшатры"
    }

    df = df.rename(columns=columns_mapping)
    columns_to_delete = ['SubLord', 'SubSubLord', 'HouseNr']
    df = df.drop(columns=columns_to_delete)
    df = translate_chart_to_russian(df)
    return df
    
def update_chart(df, filter_main_grahas=True):
    # Удалим неиспользуемые колонки, если они есть
    for col in ['LonDecDeg', 'SignLonDecDeg', 'LatDMS', 'SubLord', 'SubSubLord', 'HouseNr']:
        if col in df.columns:
            df = df.drop(columns=col)
    df = add_house_info(df)
    if filter_main_grahas:
        df = filter_main_grahas(df)
    return df

def decimal_to_dms(deg):
    d = int(deg)
    m_float = (deg - d) * 60
    m = int(m_float)
    s = int(round((m_float - m) * 60))
    return f"{d:02d}°{m:02d}'{s:02d}''"

In [ ]:
def build_navamsa_df(df):
    rows = []

    for _, row in df.iterrows():
        planet = row["Планета"]
        d1_sign = row["Знак"]
        deg_in_sign = row["SignLonDecDeg"]

        if pd.isna(deg_in_sign) or d1_sign not in D1_TO_D9_MAP:
            continue

        navamsa_index = int(deg_in_sign // (30 / 9))
        navamsa_sign = D1_TO_D9_MAP[d1_sign][navamsa_index]
        deg_in_navamsa = deg_in_sign - navamsa_index * (30 / 9)
        deg_dms = decimal_to_dms(deg_in_navamsa)

        rows.append({
            "Планета": planet,
            "Знак": navamsa_sign,
            "Градусы": deg_dms
        })

    return pd.DataFrame(rows)

In [2]:
# user data
city = 'Nicosia'
country = 'Cyprus'
birth_date = "21.05.2023"
birth_time = "01:50:00"
latitude, longitude = "35.10", "33.22"

In [3]:
df = build_chart(birth_date, birth_time, latitude, longitude)

In [4]:
df

,Планета,Знак,isRetroGrade,LonDecDeg,SignLonDMS,SignLonDecDeg,LatDMS,Накшатра,Управитель,Управитель накшатры
0,Асцендент,Водолей,None,311.308,+11:18:28,11.3078,None,Шатабхиша,Сатурн,Раху
1,Солнце,Телец,False,35.485,+05:29:07,5.4850,+00:57:46,Криттика,Венера,Солнце
2,Луна,Телец,False,50.827,+20:49:36,20.8270,+12:42:16,Рохини,Венера,Луна
3,Меркурий,Овен,False,12.952,+12:57:06,12.9520,+00:26:05,Ашвини,Марс,Кету
4,Венера,Близнецы,False,80.160,+20:09:38,20.1600,+01:02:46,Пунарвасу,Меркурий,Юпитер
5,Марс,Рак,False,95.993,+05:59:34,5.9930,+00:34:14,Пушья,Луна,Сатурн
6,Юпитер,Овен,False,6.779,+06:46:45,6.7790,+00:13:31,Ашвини,Марс,Кету
7,Сатурн,Водолей,False,312.401,+12:24:04,12.4010,+00:02:41,Шатабхиша,Сатурн,Раху
8,Уран,Овен,False,25.397,+25:23:50,25.3970,+00:03:26,Бхарани,Марс,Венера
9,Нептун,Рыбы,False,333.060,+03:03:34,3.0600,+00:01:16,Пурва Бхадрапада,Юпитер,Юпитер


In [6]:
navamsa = build_navamsa_df(df)

In [7]:
navamsa

,Планета,Знак,Градусы
0,Асцендент,Козерог,01°18'28''
1,Солнце,Водолей,02°09'06''
2,Луна,Рак,00°49'37''
3,Меркурий,Рак,02°57'07''
4,Венера,Овен,00°09'36''
5,Марс,Лев,02°39'35''
6,Юпитер,Близнецы,00°06'44''
7,Сатурн,Козерог,02°24'04''
8,Уран,Скорпион,02°03'49''
9,Нептун,Рак,03°03'36''
